In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tabula
import dill
import re
f = '/home/joe/data/covid19/20200307.pdf'
#df = tabula.read_pdf(f, pages = "all", multiple_tables = True)
#dill.dump_session('parsed.db')
dill.load_session('parsed.db')

In [12]:
# %load findChina.py
def findChina(df):
    x = 'Province/' # id China with this
    y = 'Hubei'# id China with this
    for df in tables:
        if (x in df.columns.values):
            if (y in df[x].tolist()):
                print(re.split('\\s+',df['Cumulative'].iloc[-1]))


In [10]:
findChina(df)

['80813', '3073']
